In [1]:
import pandas as pd
from pathlib import Path
from training_data_reasoning import get_reasoning_df
import json
import openai
import os
import api_calls as api
openai.api_key = os.getenv("OPENAI_API_KEY")

In [106]:
TRAINING_DATA = Path("./data/training_data/")

In [107]:
df = pd.DataFrame()
for file in TRAINING_DATA.glob("*.xlsx"):
    df = pd.concat([df, pd.read_excel(file, usecols = ["sentence", "label"])])
df.to_csv(TRAINING_DATA/"training_data.csv")

In [108]:
training_data = pd.read_csv(TRAINING_DATA/"training_data.csv")

In [109]:
test=training_data.head(200)

In [110]:
await get_reasoning_df(test, TRAINING_DATA/"training_data_annotated.csv")

2023-10-04 21:14:25 [info     ] Rate limit exceeded. Initiating Exponential backoff wait_time=1
2023-10-04 21:14:25 [info     ] Rate limit exceeded. Initiating Exponential backoff wait_time=1
2023-10-04 21:14:25 [info     ] Rate limit exceeded. Initiating Exponential backoff wait_time=1
2023-10-04 21:14:25 [info     ] Rate limit exceeded. Initiating Exponential backoff wait_time=1
2023-10-04 21:14:25 [info     ] Rate limit exceeded. Initiating Exponential backoff wait_time=1
2023-10-04 21:14:25 [info     ] Rate limit exceeded. Initiating Exponential backoff wait_time=1
2023-10-04 21:14:25 [info     ] Rate limit exceeded. Initiating Exponential backoff wait_time=1
2023-10-04 21:14:25 [info     ] Rate limit exceeded. Initiating Exponential backoff wait_time=1
2023-10-04 21:14:25 [info     ] Rate limit exceeded. Initiating Exponential backoff wait_time=1
2023-10-04 21:14:26 [info     ] Rate limit exceeded. Initiating Exponential backoff wait_time=1
2023-10-04 21:14:26 [info     ] Rate lim

CancelledError: 

In [ ]:
training_data_short = pd.read_csv(TRAINING_DATA/"training_data_annotated.csv", usecols=["sentence", "label"])

In [ ]:
system_message = "you are an economist. label the following sentence as hawkish, dovish or neutral and provide a reason for your label."
with open(TRAINING_DATA/"training_data.jsonl", "w") as training_data_file:
    for i in range(training_data_short.shape[0]):
        json.dump({"messages":[{"role":"system", "content":system_message}, {"role":"user", "content": training_data_short.iloc[i, 0]}, {"role":"assistant", "content": training_data_short.iloc[i, 1]}]}, training_data_file)
        training_data_file.write("\n")



In [ ]:
openai.File.create(
    file = open(TRAINING_DATA/"training_data.jsonl", "rb"),
    purpose = "fine-tune"
)

<File file id=file-rxK0PgouYtXWOgJ9sFw0w3Lf at 0x12e2bbce0> JSON: {
  "object": "file",
  "id": "file-rxK0PgouYtXWOgJ9sFw0w3Lf",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 132265,
  "created_at": 1696449025,
  "status": "uploaded",
  "status_details": null
}

In [ ]:
openai.FineTuningJob.create(training_file = "file-rxK0PgouYtXWOgJ9sFw0w3Lf", model = "gpt-3.5-turbo")

<FineTuningJob fine_tuning.job id=ftjob-RXiDrDIG1wzG9WrGST4jlz7K at 0x12e00c630> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-RXiDrDIG1wzG9WrGST4jlz7K",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1696449137,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-I4j7CvqT1sUoMLq85ywq27zV",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-rxK0PgouYtXWOgJ9sFw0w3Lf",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [115]:
openai.FineTuningJob.retrieve("ftjob-RXiDrDIG1wzG9WrGST4jlz7K")

<FineTuningJob fine_tuning.job id=ftjob-RXiDrDIG1wzG9WrGST4jlz7K at 0x12ed13650> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-RXiDrDIG1wzG9WrGST4jlz7K",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1696449137,
  "finished_at": 1696450537,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::862ZOMQr",
  "organization_id": "org-I4j7CvqT1sUoMLq85ywq27zV",
  "result_files": [
    "file-rDw7ikBKzRlU2M2zuYnaq4nL"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-rxK0PgouYtXWOgJ9sFw0w3Lf",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 67839,
  "error": null
}

In [4]:
await api.get_api_call([{"role":"system", "content": "you are an economist. label the following sentence as hawkish, dovish or neutral and provide a reason for your label."},{"role":"user", "content":"The slowdown would reflect factors that were expected to damp the growth of overall business investment spending and a greater saturation of potential computer markets that might lead to more emphasis on replacement demand rather than the further expansion of capacity."}])

2023-10-04 21:21:31 [info     ] got response                   response_status=200


'This sentence is neutral because it simply describes potential economic factors without expressing any judgment or opinion on US monetary policy.'